In [1]:
!pip install h5py


   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.8/2.9 MB 2.6 MB/s eta 0:00:01
   ------------------ --------------------- 1.3/2.9 MB 2.6 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.9 MB 2.9 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 2.9 MB/s eta 0:00:00


In [5]:
import h5py

# 1.1 Dosyayı aç ve ana grupları listele
hf = h5py.File('fanet_topo_dataset.h5', 'r')
print("Ana gruplar:", list(hf.keys()))  # -> ['fanet_topo_dataset']

grp = hf['fanet_topo_dataset']
snapshots = list(grp.keys())
print("Toplam snapshot:", len(snapshots))
print("Örnek snapshot isimleri:", snapshots[:5])

# 1.2 İlk snapshot'ın içeriğini göster
snap0 = grp[snapshots[0]]
print("positions shape:", snap0['positions'].shape)
print("persistence_image length:", snap0['persistence_image'].shape)
print("Attrs:")
for k, v in snap0.attrs.items():
    print(f"  {k} = {v}")

hf.close()



Ana gruplar: ['fanet_topo_dataset']


RuntimeError: Unable to get group info (addr overflow, addr = 14535688, size = 544, eoa = 2048)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 2.1 Tüm snapshot’lardaki verileri oku
hf = h5py.File('fanet_topo_dataset.h5', 'r')
grp = hf['fanet_topo_dataset']

X_img = []
X_feat = []
y = []

for name in grp:
    s = grp[name]
    # persistence image
    img = s['persistence_image'][:]
    X_img.append(img)
    # sayısal öznitelikler
    bf = s.attrs['beta0_fixed']
    ba = s.attrs['beta0_adaptive']
    rc = s.attrs['r_c_adaptive']
    X_feat.append([bf, ba, rc])
    # etiket (örneğin RWP vs GM sınıflandırması için)
    model = s.attrs['model'].decode() if isinstance(s.attrs['model'], bytes) else s.attrs['model']
    y.append(0 if model=='RWP' else 1)

hf.close()

X_img = np.vstack(X_img)        # (num_snapshots, pixels* pixels)
X_feat = np.array(X_feat)       # (num_snapshots, 3)
y = np.array(y)                 # (num_snapshots,)

# 2.2 Normalize / standardize
img_scaler = MinMaxScaler()
X_img = img_scaler.fit_transform(X_img)

feat_scaler = StandardScaler()
X_feat = feat_scaler.fit_transform(X_feat)

# 2.3 Train/Val/Test böl
X_img_tr, X_img_temp, X_feat_tr, X_feat_temp, y_tr, y_temp = train_test_split(
    X_img, X_feat, y, test_size=0.3, random_state=42, stratify=y)

X_img_val, X_img_te, X_feat_val, X_feat_te, y_val, y_te = train_test_split(
    X_img_temp, X_feat_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Shapes:", X_img_tr.shape, X_img_val.shape, X_img_te.shape)



Shapes: (350000, 400) (75000, 400) (75000, 400)


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class FANETDataset(Dataset):
    def __init__(self, X_img, X_feat, y):
        self.X_img = torch.from_numpy(X_img).float()
        self.X_feat = torch.from_numpy(X_feat).float()
        self.y = torch.from_numpy(y).float().unsqueeze(1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return {
            'img': self.X_img[idx],      # persistence image vector
            'feat': self.X_feat[idx],    # numeric features
            'y': self.y[idx]             # label
        }

# Dataset objeleri
train_ds = FANETDataset(X_img_tr, X_feat_tr, y_tr)
val_ds   = FANETDataset(X_img_val, X_feat_val, y_val)
test_ds  = FANETDataset(X_img_te, X_feat_te, y_te)

# DataLoader’lar
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)



In [11]:
import torch.nn as nn

class BaselineMLP(nn.Module):
    def __init__(self, img_dim, feat_dim, hidden=64):
        super().__init__()
        self.fc1 = nn.Linear(img_dim + feat_dim, hidden)
        self.fc2 = nn.Linear(hidden, 1)

    def forward(self, img, feat):
        x = torch.cat([img, feat], dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data


class FANET_TopoGNN(nn.Module):
    def __init__(self, img_dim, feat_dim, hidden=64, gc_hidden=32, fusion_dim=128):
        super().__init__()
        # (a) GCN structural encoder
        self.conv1 = pyg_nn.GCNConv(2, gc_hidden)
        self.conv2 = pyg_nn.GCNConv(gc_hidden, gc_hidden)
        # (b) PI encoder
        self.pi_mlp = nn.Sequential(
            nn.Linear(img_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden)
        )
        # (c) Gated fusion
        self.proj_struct = nn.Linear(gc_hidden, fusion_dim)
        self.proj_pi     = nn.Linear(hidden, fusion_dim)
        self.proj_gate   = nn.Linear(fusion_dim*2, fusion_dim)
        # (d) Regression head
        self.head = nn.Sequential(
            nn.Linear(fusion_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, data, img, feat):
        # --- structural: GCN on input graph in data ---
        x, edge_index = data.x, data.edge_index
        h = F.relu(self.conv1(x, edge_index))
        h = F.relu(self.conv2(h, edge_index))
        z_struct = pyg_nn.global_mean_pool(h, data.batch)  # (batch, gc_hidden)

        # --- topological: PI mlp ---
        z_pi = self.pi_mlp(img)  # (batch, hidden)

        # --- gated fusion ---
        s = self.proj_struct(z_struct)
        p = self.proj_pi(z_pi)
        gate = torch.sigmoid(self.proj_gate(torch.cat([s,p], dim=1)))
        z_f = gate * s + (1-gate) * p

        # --- regression ---
        return self.head(z_f)


In [3]:
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FANET_TopoGNN(img_dim=X_img.shape[1], feat_dim=X_feat.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = nn.MSELoss()

best_val_loss = float('inf')
patience = 10
counter = 0

for epoch in range(1, 101):
    # --- train ---
    model.train()
    total_loss = 0
    for batch in train_loader:
        img = batch['img'].to(device)
        feat= batch['feat'].to(device)
        y   = batch['y'].to(device)
        # dummy graph data; burayı gerçek edge_index ile değiştirin
        data = Data(x=torch.randn(len(y),2).to(device),
                    edge_index=torch.empty((2,0),dtype=torch.long).to(device),
                    batch=torch.arange(len(y), device=device))
        optimizer.zero_grad()
        y_hat = model(data, img, feat)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(y)
    total_loss /= len(train_ds)

    # --- validate ---
    model.eval()
    val_preds, val_trues = [], []
    with torch.no_grad():
        for batch in val_loader:
            img = batch['img'].to(device)
            feat= batch['feat'].to(device)
            y   = batch['y'].to(device)
            data = Data(x=torch.randn(len(y),2).to(device),
                        edge_index=torch.empty((2,0),dtype=torch.long).to(device),
                        batch=torch.arange(len(y), device=device))
            y_hat = model(data, img, feat)
            val_preds.append(y_hat.cpu().numpy())
            val_trues.append(y.cpu().numpy())
    val_preds = np.vstack(val_preds)
    val_trues = np.vstack(val_trues)
    val_loss = mean_squared_error(val_trues, val_preds)

    print(f"Epoch {epoch}: Train MSE={total_loss:.4f}, Val MSE={val_loss:.4f}")
    # early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping.")
            break



NameError: name 'X_img' is not defined